**About** : This notebook is used to prepare the data.

### Packages

In [ ]:
!pip install -q ../output/nvidia_dali_nightly_cuda110-1.23.0.dev20230203-7187866-py3-none-manylinux2014_x86_64.whl

In [ ]:
# YoloX
!pip install --no-index -qqq ../output/rsna-yolox/loguru-0.6.0-py3-none-any.whl
!pip install --no-index -qqq ../output/rsna-yolox/thop-0.1.1.post2209072238-py3-none-any.whl
!pip install --no-index -qqq ../output/rsna-yolox/protobuf-3.20.3-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl
!pip install --no-index -qqq ../output/rsna-yolox/onnx-1.13.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install --no-index -qqq ../output/rsna-yolox/onnx_simplifier-0.4.10-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install --no-index -qqq ../output/rsna-yolox/Cython-0.29.33-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl

!pip install -qqq ../output/rsna-yolox/pycocotools-2.0.6-cp37-cp37m-linux_x86_64.whl

!mkdir -p /tmp/pip/cache/

!cp ../output/rsna-yolox/protobuf-3.20.3-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl /tmp/pip/cache/
!cp ../output/rsna-yolox/onnx-1.13.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl /tmp/pip/cache/
!cp ../output/rsna-yolox/onnx_simplifier-0.4.10-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl /tmp/pip/cache/
!cp ../output/rsna-yolox/onnx_simplifier-0.4.10-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl /tmp/pip/cache/
!cp ../output/rsna-yolox/tensorboard_plugin_wit-1.8.1-py3-none-any.whl /tmp/pip/cache/
!cp ../output/rsna-yolox/tensorboard_data_server-0.6.1-py3-none-manylinux2010_x86_64.whl /tmp/pip/cache/
!cp ../output/rsna-yolox/tensorboard-2.11.2-py3-none-any.whl /tmp/pip/cache/
!cp ../output/rsna-yolox/Werkzeug-2.2.2-py3-none-any.whl /tmp/pip/cache/
!cp ../output/rsna-yolox/google_auth-2.16.0-py2.py3-none-any.whl /tmp/pip/cache/
!cp ../output/rsna-yolox/google_auth_oauthlib-0.4.6-py2.py3-none-any.whl /tmp/pip/cache/
!cp ../output/rsna-yolox/absl_py-1.4.0-py3-none-any.whl /tmp/pip/cache/
!cp ../output/rsna-yolox/rsa-4.9-py3-none-any.whl /tmp/pip/cache/
!cp ../output/rsna-yolox/pyasn1_modules-0.2.8-py2.py3-none-any.whl /tmp/pip/cache/
!cp ../output/rsna-yolox/grpcio-1.51.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl /tmp/pip/cache/

!pip install -qqq ../output/rsna-yolox/tensorboard-2.11.2-py3-none-any.whl --find-links /tmp/pip/cache/

!rm -rf /tmp/pip/cache/YoloX
!cp -r ../output/rsna-yolox/YOLOX /tmp/pip/cache/
!cd /tmp/pip/cache/YOLOX; pip install -qqq --find-links /tmp/pip/cache/ -v -e . 

### Images

In [ ]:
# Add UINT16 on Pytorch for feed_ndarray with DALI
with open("/usr/local/lib/python3.8/dist-packages/nvidia/dali/plugin/pytorch.py", "r") as f:
    content = f.read()

content_uint16 = content.replace(
    "    types.DALIDataType.INT8:    torch.int8,",
    "    types.DALIDataType.INT8:    torch.int8,\n    types.DALIDataType.UINT16:  torch.int16,"
)

with open("/usr/local/lib/python3.8/dist-packages/nvidia/dali/plugin/pytorch.py", "w") as f:
    f.write(content_uint16)

In [ ]:
import torch
import pydicom
import dicomsdl
import numpy as np
import nvidia.dali.types as types
from pydicom.filebase import DicomBytesIO
from nvidia.dali.experimental import eager
from nvidia.dali.types import DALIDataType
from nvidia.dali.plugin.pytorch import feed_ndarray


def dicomsdl_to_numpy_image(dicom, index=0):
    info = dicom.getPixelDataInfo()
    dtype = info["dtype"]
    if info["SamplesPerPixel"] != 1:
        raise RuntimeError("SamplesPerPixel != 1")
    else:
        shape = [info["Rows"], info["Cols"]]
    outarr = np.empty(shape, dtype=dtype)
    dicom.copyFrameData(index, outarr)
    return outarr


def load_img_dicomsdl(f):
    return dicomsdl_to_numpy_image(dicomsdl.open(f))


def read_normalize_basic(dcm_path):
    dicom = pydicom.dcmread(dcm_path, stop_before_pixels=True)

    if dicom.file_meta.TransferSyntaxUID == "1.2.840.10008.1.2.4.90":
        to_find = b"\x00\x00\x00\x0C"
    elif dicom.file_meta.TransferSyntaxUID == '1.2.840.10008.1.2.4.70':
        to_find = b"\xff\xd8\xff\xe0"
    else:
        to_find = None

    if to_find is not None:
        with open(dcm_path, "rb") as fp:
            raw = DicomBytesIO(fp.read())
            ds = pydicom.dcmread(raw)
        offset = ds.PixelData.find(to_find)
        bitstream = np.array(bytearray(ds.PixelData[offset:]), dtype=np.uint8)
        # Decode image to GPU
        img = eager.experimental.decoders.image(
            [bitstream],
            device="gpu",
            output_type=types.ANY_DATA,
            dtype=DALIDataType.UINT16,
        )[0]
        # Wrap to Pytorch
        img_torch = torch.empty(img.shape(), dtype=torch.int16, device="cuda")
        feed_ndarray(img, img_torch, cuda_stream=torch.cuda.current_stream(device=0))
        img = img_torch.float().squeeze()

    else:
        # read on cpu then load
        try:
            img = load_img_dicomsdl(dcm_path)
        except NotImplementedError:
            img = pydicom.dcmread(dcm_path, stop_before_pixels=False).pixel_array
        img = torch.from_numpy(img.astype(np.float32))
        img = img.to("cuda")

    # Normalize image to 0 - 1
    min_, max_ = img.min(), img.max()
    img = (img - min_) / (max_ - min_ + 1e-6)

    if dicom.PhotometricInterpretation == "MONOCHROME1":
        img = 1 - img

    return img

### YoloX

In [ ]:
%%writefile /tmp/pip/cache/YOLOX/exps/example/custom/nano.py

import os
import torch.nn as nn
from yolox.exp import Exp as MyExp


class Exp(MyExp):
    def __init__(self):
        super(Exp, self).__init__()
        self.depth = 0.33
        self.width = 0.25
        
        self.mosaic_prob = 0.0
        self.mixup_prob = 0.0
        self.hsv_prob = 0.0
        self.degrees = 2.0
        self.translate = 0.05
        self.shear = 0.0
        self.multiscale_range = 3 # 5
        self.random_size = (20, 34)
        self.mosaic_scale = (0.5, 1.5)        
        self.input_size = (1024, 1024)
        self.test_size = (1024, 1024)
                
        self.exp_name = os.path.split(os.path.realpath(__file__))[1].split(".")[0]
        self.enable_mixup = False

        # Define yourself dataset path
        self.data_dir = "datasets/COCO"
        self.train_ann = "instances_train1024.json"
        self.val_ann = "instances_valid1024.json"

        self.warmup_epochs = 3
        self.num_classes = 1
        self.max_epoch = 54 # 300
        self.seed = 42
        

    def get_model(self, sublinear=False):

        def init_yolo(M):
            for m in M.modules():
                if isinstance(m, nn.BatchNorm2d):
                    m.eps = 1e-3
                    m.momentum = 0.03
        if "model" not in self.__dict__:
            from yolox.models import YOLOX, YOLOPAFPN, YOLOXHead
            in_channels = [256, 512, 1024]
            # NANO model use depthwise = True, which is main difference.
            backbone = YOLOPAFPN(self.depth, self.width, in_channels=in_channels, depthwise=True)
            head = YOLOXHead(self.num_classes, self.width, in_channels=in_channels, depthwise=True)
            self.model = YOLOX(backbone, head)

        self.model.apply(init_yolo)
        self.model.head.initialize_biases(1e-2)
        return self.model

In [ ]:
import os
import sys
import torch
import importlib
import numpy as np
import pandas as pd
import torch.nn.functional as F

from torch.cuda import amp

import sys
sys.path.append("/tmp/pip/cache/YOLOX/")
sys.path.append("../output/rsna-yolox/YOLOX/")

from yolox.utils import get_model_info, postprocess
from yolox_utils import PadIfNeeded, scale_boxes, preprocess


def init_roi(device_roi_='cuda:0'):
    # To avoid name '__file__' is not defined
    exp_file = "/tmp/pip/cache/YOLOX/exps/example/custom/nano.py"
    sys.path.append(os.path.dirname(exp_file))
    current_exp = importlib.import_module(os.path.basename(exp_file).split(".")[0])
    exp = current_exp.Exp()
    exp.test_conf = 0.0
    exp.test_size = (1024, 1024)
    exp.nmsthre = 0.45  # 0.65
    model_roi_ = exp.get_model()
    print("Model Summary: {}".format(get_model_info(model_roi_, exp.test_size)))
    model_roi_ = model_roi_.half().eval().to(device_roi_)
    ckpt_file = (
        "../output/rsna-yolox/nano/best_ckpt_v2.pth"  # mAP@(0.50:0.95)=0.9625
    )
    print("Loading YoloX weights", ckpt_file)
    ckpt = torch.load(ckpt_file, map_location=device_roi_)
    model_roi_.load_state_dict(ckpt["model"])
    model_roi_.max_det = 1
    model_roi_.nmsthre = exp.nmsthre
    model_roi_.test_conf = exp.test_conf
    model_roi_.test_size = exp.test_size
    model_roi_.num_classes = 1
    model_roi_.stride = 64
    model_roi_.amp = True  # FP16
    return model_roi_


device_roi = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  # 'cpu'
model_roi = None


class Config:
    IMAGE_BOX_SIZE_H = 1024
    IMAGE_BOX_SIZE_W = 1024

    image_pin_tr = PadIfNeeded(
        min_height=IMAGE_BOX_SIZE_H, min_width=IMAGE_BOX_SIZE_W
    )  # Pad to 1

    YOLO_PREPROCESS_GPU = True
    YOLO_SIZE = 1024


def gpu_roi_crop(img, model_roi, config):
    if model_roi is None:
        raise "Missing ROI model"
    # dt = (Profile(),Profile(),Profile())
    # h, w = img.size(0), img.size(1)
    # row_pd = pd.DataFrame([(h, w, -1, f)], columns=["h", "w", "confidence", "dcm_path"])
    row_pd = None

    if config.YOLO_PREPROCESS_GPU is False:
        # CPU, GPU round trip
        results = model_roi(
            (img * 255).squeeze(2).cpu().numpy().astype(np.uint8), size=config.YOLO_SIZE
        )
        row_pd = results.pandas().xyxy[0]
    else:
        # Full GPU
        ims = [img.squeeze(2)]
        imgs, files, shape0, shape1 = preprocess(
            ims, size=config.YOLO_SIZE, model_stride=model_roi.stride
        )  # Resize/Pad
        img_roi = imgs[0]
        # YoloX preprocessing: Pad to 1024x1024 (bgcolor=114., 0-255.)
        # print("ROI img", img_roi.shape, img_roi.max())
        ratio = min(
            model_roi.test_size[0] / img_roi.shape[0],
            model_roi.test_size[1] / img_roi.shape[1],
        )
        padding = config.image_pin_tr(img_roi.shape[0], img_roi.shape[1])
        img_roi = F.pad(
            input=img_roi * 255.0,
            pad=(
                padding.get("pad_left"),
                padding.get("pad_right"),
                padding.get("pad_top"),
                padding.get("pad_bottom"),
            ),
            mode="constant",
            value=114.0,
        )
        img_roi = (
            img_roi.expand((3, img_roi.shape[0], img_roi.shape[1])).unsqueeze(0).float()
        )  # (BCHW) float32
        # print("YoloX re-padding completed", img_roi.shape, img_roi.max())
        with amp.autocast(model_roi.amp and (img_roi.device.type != "cpu")):
            with torch.no_grad():
                # y = model_roi(img_roi, size=YOLO_SIZE).cpu()
                y = model_roi(img_roi)
                y = postprocess(
                    y,
                    model_roi.num_classes,
                    model_roi.test_conf,
                    model_roi.nmsthre,
                    class_agnostic=True,
                )[0]
                y = y.cpu().numpy()
                bboxes = y[:, 0:4]
                bboxes /= ratio
                scores = y[:, 4] * y[:, 5]
                row_pd = pd.DataFrame(bboxes, columns=["xmin", "ymin", "xmax", "ymax"])
                row_pd["score"] = scores
                # Keep best bbox
                row_pd = (
                    row_pd.sort_values(["score"], ascending=False)
                    .reset_index(drop=True)
                    .iloc[0: model_roi.max_det, :]
                )

        # Unpad, scale back and clip if needed
        row_pd["xmin"] = row_pd["xmin"] - padding.get("pad_left")
        row_pd["xmax"] = row_pd["xmax"] - padding.get("pad_right")
        row_pd["ymin"] = row_pd["ymin"] - padding.get("pad_top")
        row_pd["ymax"] = row_pd["ymax"] - padding.get("pad_bottom")
        y = np.expand_dims(row_pd[["xmin", "ymin", "xmax", "ymax"]].values, axis=0)
        scale_boxes(shape1, y[0][:, :4], shape0[0])
        row_pd = pd.DataFrame(y[0], columns=["xmin", "ymin", "xmax", "ymax"])

    if len(row_pd) == 1:
        x1, x2 = int(row_pd["xmin"]), int(row_pd["xmax"])
        y1, y2 = int(row_pd["ymin"]), int(row_pd["ymax"])
        # Crop image
        img = img[y1:y2, x1:x2]

    return img, (y1, y2, x1, x2)

## Main

In [ ]:
import cv2
import glob
import torch
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
df_train = pd.read_csv('../input/train.csv')
df_train["image_name"] = df_train[["patient_id", "image_id"]].apply(
    lambda x : str(x[0]) + "/" + str(x[1]) + ".dcm", axis=1
)

DATA_PATH = "/workspace/"
PATH_TO_DCMS = DATA_PATH + "data_rsna/train_images/"

In [ ]:
IMG_PATH = "/kaggle/input/rsna-breast-cancer-detection/train_images/"
test_images = glob.glob(f"{IMG_PATH}*/*.dcm")

In [ ]:
# SIZE = (1024 + 512, 1024)
# SIZE_SUFFIX = "_".join(map(str, list(SIZE))) if SIZE is not None else ""

# SAVE_FOLDER = f"../input/yolox_{SIZE_SUFFIX}/"

# files = df_train.image_name

# print(f'Saving images to {SAVE_FOLDER}')
# os.makedirs(SAVE_FOLDER, exist_ok=True)

In [ ]:
SIZE = ((1024) * 3, (1024) * 2)
SIZE_SUFFIX = "_".join(map(str, list(SIZE))) if SIZE is not None else ""

In [ ]:
SAVE_FOLDER = f"../input/yolox_{SIZE_SUFFIX}/"

files = df_train.image_name

print(f'Saving images to {SAVE_FOLDER}')
os.makedirs(SAVE_FOLDER, exist_ok=True)

In [ ]:
# files = glob.glob('../input/vindr-mammo/*.dicom')
# PATH_TO_DCMS = ""

# SAVE_FOLDER = f"../input/vindr_yolox_{SIZE_SUFFIX}/"

# print(f'Saving images to {SAVE_FOLDER}')
# os.makedirs(SAVE_FOLDER, exist_ok=True)

In [ ]:
# files = glob.glob('../input/cbis-ddcm/*.dcm')
# PATH_TO_DCMS = ""

# SAVE_FOLDER = f"../input/cbis_yolox_{SIZE_SUFFIX}/"

# print(f'Saving images to {SAVE_FOLDER}')
# os.makedirs(SAVE_FOLDER, exist_ok=True)

In [ ]:
# files = glob.glob('../input/CMMD/*/*/*/*.dcm')
# PATH_TO_DCMS = ""

# SAVE_FOLDER = f"../input/cmmd_yolox_{SIZE_SUFFIX}/"

# print(f'Saving images to {SAVE_FOLDER}')
# os.makedirs(SAVE_FOLDER, exist_ok=True)

In [ ]:
files = sorted(glob.glob('../input/pasm/*/*/*.dcm')) + sorted(glob.glob('../input/pasm/*/*/*.DCM'))
PATH_TO_DCMS = ""

SAVE_FOLDER = f"../input/pasm_yolox_{SIZE_SUFFIX}/"

print(f'Saving images to {SAVE_FOLDER}')
os.makedirs(SAVE_FOLDER, exist_ok=True)

In [ ]:
%%time
model_roi = init_roi("cuda:0")

for i, file in enumerate(tqdm(files)):    
    if "cbis" in SAVE_FOLDER:
        name = file.split("/")[-1][:-4]
    elif "cmmd" in SAVE_FOLDER:
        dcm = pydicom.read_file(PATH_TO_DCMS + file, stop_before_pixels=True)
        image = file.split("/")[-1][:-4]
        patient = file.split("/")[-4]
        name = f"{patient}_{dcm.ImageLaterality}_{image}"
    elif "pasm" in SAVE_FOLDER:
#         dcm = pydicom.read_file(PATH_TO_DCMS + file, stop_before_pixels=True)
        image = file.split("/")[-1][:-4]
        patient = file.split("/")[-3] + "_" + file.split("/")[-2]
        name = f"{patient}_{image}"
    else:
        patient = file.split("/")[-2]
        image = file.split("/")[-1][:-4]
        name = f"{patient}_{image}"
    
    if os.path.exists(SAVE_FOLDER + f"{name}.png"):
        continue
    img_main = read_normalize_basic(PATH_TO_DCMS + file)

    try:
        img_main, (y1, y2, x1, x2) = gpu_roi_crop(img_main.unsqueeze(2), model_roi, Config)
    except:
        print(f'Error ! with image {name}')

    if SIZE is not None:
        img_main = F.interpolate(
            img_main.view(1, 1, img_main.size(0), img_main.size(1)),
            SIZE,
            mode="bilinear"
        )[0, 0]
        
    cv2.imwrite(SAVE_FOLDER + f"{name}.png", (img_main.cpu().squeeze().numpy() * 255).astype(np.uint8))

    if i % 2000 == 0:
        plt.imshow(img_main.cpu().squeeze().numpy(), cmap="gray")
        plt.axis(False)
        plt.show()

#     break

Done !